# Extract Data

In [1]:
# Dependencies
import requests
import pandas as pd

In [2]:
# Get  open WHO API for relevent information
url = "https://ghoapi.azureedge.net/api/WSH_WATER_BASIC"

In [3]:
# Loop through the list of countries and get all JSON info and put into pandas df
response = requests.get(url).json() 
   
data_df = pd.DataFrame(response['value'])
data_df.head()

,Id,IndicatorCode,SpatialDimType,SpatialDim,TimeDimType,TimeDim,Dim1Type,Dim1,Dim2Type,Dim2,...,DataSourceDim,Value,NumericValue,Low,High,Comments,Date,TimeDimensionValue,TimeDimensionBegin,TimeDimensionEnd
0,21141924,WSH_WATER_BASIC,COUNTRY,AFG,YEAR,2000,RESIDENCEAREATYPE,RUR,None,None,...,None,22,21.61913,None,None,None,2019-07-11T14:12:51.13+02:00,2000,2000-01-01T00:00:00+01:00,2000-12-31T00:00:00+01:00
1,21141925,WSH_WATER_BASIC,COUNTRY,AFG,YEAR,2000,RESIDENCEAREATYPE,TOTL,None,None,...,None,28,27.77190,None,None,None,2019-07-11T14:12:51.163+02:00,2000,2000-01-01T00:00:00+01:00,2000-12-31T00:00:00+01:00
2,21141926,WSH_WATER_BASIC,COUNTRY,AFG,YEAR,2000,RESIDENCEAREATYPE,URB,None,None,...,None,49,49.48745,None,None,None,2019-07-11T14:12:51.18+02:00,2000,2000-01-01T00:00:00+01:00,2000-12-31T00:00:00+01:00
3,21141927,WSH_WATER_BASIC,COUNTRY,AFG,YEAR,2001,RESIDENCEAREATYPE,RUR,None,None,...,None,22,21.61913,None,None,None,2019-07-11T14:12:51.21+02:00,2001,2001-01-01T00:00:00+01:00,2001-12-31T00:00:00+01:00
4,21141928,WSH_WATER_BASIC,COUNTRY,AFG,YEAR,2001,RESIDENCEAREATYPE,TOTL,None,None,...,None,28,27.79726,None,None,None,2019-07-11T14:12:51.227+02:00,2001,2001-01-01T00:00:00+01:00,2001-12-31T00:00:00+01:00


# Transform Data

In [4]:
# Create a filtered dataframe from specific columns
cleanWater_columns = ["SpatialDim", "TimeDim", "Dim1", "NumericValue"]
cleanWater_transformed = data_df[cleanWater_columns].copy()
cleanWater_transformed.head()

,SpatialDim,TimeDim,Dim1,NumericValue
0,AFG,2000,RUR,21.61913
1,AFG,2000,TOTL,27.77190
2,AFG,2000,URB,49.48745
3,AFG,2001,RUR,21.61913
4,AFG,2001,TOTL,27.79726


In [5]:
# Remove rural and urban values from dataframe
cleanWater_total = cleanWater_transformed.loc[cleanWater_transformed['Dim1'] == "TOTL"]
cleanWater_total

,SpatialDim,TimeDim,Dim1,NumericValue
1,AFG,2000,TOTL,27.77190
4,AFG,2001,TOTL,27.79726
7,AFG,2002,TOTL,29.90076
10,AFG,2003,TOTL,32.00507
13,AFG,2004,TOTL,34.12623
...,...,...,...,...
10462,ZWE,2013,TOTL,65.97417
10465,ZWE,2014,TOTL,65.47338
10468,ZWE,2015,TOTL,64.98583
10471,ZWE,2016,TOTL,64.51162


In [8]:
# Remove all years except 2017 from dataframe
cleanWater_totalYear = cleanWater_total.loc[cleanWater_total['TimeDim'] == 2017]
cleanWater_totalYear

,SpatialDim,TimeDim,Dim1,NumericValue
52,AFG,2017,TOTL,67.06462
106,ALB,2017,TOTL,91.03923
160,DZA,2017,TOTL,93.55589
214,AND,2017,TOTL,100.00000
268,AGO,2017,TOTL,55.84290
...,...,...,...,...
10258,VEN,2017,TOTL,95.72371
10312,VNM,2017,TOTL,94.71880
10366,YEM,2017,TOTL,63.47347
10420,ZMB,2017,TOTL,59.96376


In [9]:
# Remove Dim1 column
cleanWater_total_columns = ["SpatialDim", "TimeDim", "NumericValue"]
cleanWater_df = cleanWater_totalYear[cleanWater_total_columns].copy()
cleanWater_df.head()

,SpatialDim,TimeDim,NumericValue
52,AFG,2017,67.06462
106,ALB,2017,91.03923
160,DZA,2017,93.55589
214,AND,2017,100.00000
268,AGO,2017,55.84290


In [10]:
#Rename columns to be PostgreSQL compatible
cleanWater_df.columns = ["country", "year", "accessibility_percentage"]
cleanWater_df.head()

,country,year,accessibility_percentage
52,AFG,2017,67.06462
106,ALB,2017,91.03923
160,DZA,2017,93.55589
214,AND,2017,100.00000
268,AGO,2017,55.84290


# Save cleaned dataframe as csv file

In [11]:
cleanWater_df.to_csv('water.csv',index = False, header=True)